# Web Scraping from Statistical Atlas

### packages needed

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

this is the root URL for the website, which will be used multiple times in following codes

In [2]:
root = "https://statisticalatlas.com"

## Main codes

In [3]:
# get all county url, default as under category "Educational-Attainment"
def get_url_list(replacement = "/Educational-Attainment"):
    first_page = root + "/state/Pennsylvania/Overview"
    html = requests.get(first_page)
    soup = BeautifulSoup(html.content,'html.parser')
    counties = soup.select("div.info-table-contents-div a")
    url_list = []
    for county in counties:
        url = county['href']
        url = url.replace("/Overview",replacement)
        if url.startswith("/county"):
            url = root+url
            url_list.append(url)
    url_list = list(set(url_list))
    return url_list

In [4]:
# for counties main page, the prefix is "/Overview"
counties = get_url_list("/Overview")

In [26]:
# get all zipcode url, and responding county
def get_zipcode_url():
    county_zipcode = []
    zipcode_url = []
    for county in counties:
        html = requests.get(county)
        soup = BeautifulSoup(html.content, 'html.parser')
        zipcodes = soup.select("div.info-table-contents-td.col-sm-9 div a")
        for zipcode in zipcodes:
            if zipcode['href'].startswith("/zip"):
                a = zipcode['href']
                a = a.replace("Overview","Educational-Attainment")
                zipcode_url.append(root+a)
                a = [county.split("/")[5],zipcode['href'].split("/")[2]]
                county_zipcode.append(a)
    return zipcode_url, county_zipcode

In [6]:
# get the tags we need for the data under one county
def get_soup(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,'html.parser')
    results = soup.select("div.figure-container svg g g title")
    return results

In [7]:
# get educational attainement information for each county 
# parameter is the result returned by get_soup()
def get_each_Educational_Attainment(results):
    result = []
    i = 0
    for r in results:
        if i < 3 and "%" in r.text.strip():
            result.append(r.text.strip())
            i = i + 1
    if len(result) is not 0:
        map = {"Higher Degree": result[0],"H.S. Diploma": result[1],"No H.S. Diploma": result[2]}
    else:
        map = {"Higher Degree": "Not applicable","H.S. Diploma": "Not applicable","No H.S. Diploma": "Not applicable"}
    return(map)

In [9]:
# get educational attainment information for all counties
def get_all_Eductional_Attainment(url_list,index = 5):
    i=0
    headers = ['County','Higher Degree','H.S. Diploma','No H.S. Diploma']
    df = pd.DataFrame(columns=headers)
    for a_list in url_list:
        county_name = a_list.split("/")[index]
        print(county_name + ":")
        valmap = get_each_Educational_Attainment(get_soup(a_list))
        print(valmap)
        print("\n")
        county = [county_name]
        mapvalue = list(valmap.values())
        combList = county + mapvalue
        df.loc[i] = combList
        i += 1
    return df

In [10]:
# get detailed educational attainment information for each county
# parameter is result returned from get_soup()
def get_each_Detailed_Educational_Attainment(results):
    result = []
    i = 0
    for r in results:
        if i < 12 and "%" in r.text.strip():
            i = i + 1
        elif i < 22 and "%" in r.text.strip():
            result.append(r.text.strip())
            i = i + 1
    if len(result) is not 0:
        map = {"Doctorate": result[0],"Professional": result[1],"Master's": result[2],
       "Bachelor's":result[3],"Associate's":result[4],"Some College":result[5],
       "High School": result[6],"Some H.S.":result[7],"Less than H.S.":result[8],
       "None":result[9]}
    else:
        map = {"Doctorate": "Not applicable","Professional": "Not applicable","Master's": "Not applicable",
       "Bachelor's":"Not applicable","Associate's":"Not applicable","Some College":"Not applicable",
       "High School": "Not applicable","Some H.S.":"Not applicable","Less than H.S.":"Not applicable",
       "None":"Not applicable"}
    return(map)

In [11]:
# get detailed eudcational attainment for all counties
def get_all_Detailed_Educational_Attainment(url_list, index = 5):
    i=0
    headers = ['County','Doctorate','Professional','Masters','Bachelor','Associate','Some College','High School','Some HS','Less than HS','None']
    df = pd.DataFrame(columns=headers)
    for a_list in url_list:
        county_name = a_list.split("/")[index]
        print(county_name + ":")
        valmap = get_each_Detailed_Educational_Attainment(get_soup(a_list))
        print(valmap)
        print("\n")
        county = [county_name]
        mapvalue = list(valmap.values())
        combList = county + mapvalue
        df.loc[i] = combList
        i += 1
    return df

In [12]:
# get eductaional attainment information regarding sex for each county
# parameter is the result returned from get_soup()
def get_each_Educational_Attainment_Sex_Ratio(results):
    result = []
    i = 0
    for r in results:
        if i < 53 and "%" in r.text.strip():
            i = i + 1
        elif i < 61 and "%" in r.text.strip():
            if i == 52 or i == 55 or i ==58:
                i = i + 1
            else:
                result.append(r.text.strip())
                i = i + 1
    if len(result) is not 0:
        map = {"Higher Degree (Female)": result[0],"Higher Degree (Male)": result[1],"H.S. Diploma (Female)": result[2],
       "H.S. Diploma (Male)":result[3],"No H.S. Diploma (Female)":result[4],"No H.S. Diploma (Male)":result[5]}
    else:
        map = {"Higher Degree (Female)": "Not applicable","Higher Degree (Male)": "Not applicable","H.S. Diploma (Female)": "Not applicable",
       "H.S. Diploma (Male)":"Not applicable","No H.S. Diploma (Female)":"Not applicable","No H.S. Diploma (Male)":"Not applicable"}
    return(map)

In [13]:
# get eductaional attainment information regard sex for all counties
def get_all_Educational_Attainment_Sex_Ratio(url_list, index = 5):
    i = 0
    headers = ['County', 'Higher Degree (Female)', 'Higher Degree (Male)', 'H.S. Diploma (Female)',
               'H.S. Diploma (Male)', 'No H.S. Diploma (Female)', 'No H.S. Diploma (Male)']
    df = pd.DataFrame(columns=headers)
    for a_list in url_list:
        county_name = a_list.split("/")[index]
        print(county_name + ":")
        valmap = get_each_Educational_Attainment_Sex_Ratio(get_soup(a_list))
        print(valmap)
        print()
        county = [county_name]
        mapvalue = list(valmap.values())
        combList = county + mapvalue
        df.loc[i] = combList
        i += 1
    return df

In [14]:
# get detailed eductaional attainment information regarding sex for each county
# parameter is the result returned from get_soup()
def get_each_Detailed_Educational_Attainment_Sex_Ratio(results):
    male_result = []
    female_result = []
    i = 0
    for r in results:
        if i < 62 and "%" in r.text.strip():
            i = i + 1
        elif i%3 == 2 and "%" in r.text.strip():
            female_result.append(r.text.strip())
            i = i + 1
        elif i%3 == 0 and "%" in r.text.strip():
            male_result.append(r.text.strip())
            i = i + 1
        elif "%" in r.text.strip():
            i = i + 1
    if len(male_result) is not 0 and len(female_result) is not 0:
        map = {"Female":{"Doctorate": female_result[0],"Professional": female_result[1],
        "Master's": female_result[2],"Bachelor's":female_result[3],"Associate's":female_result[4],
        "Some College":female_result[5],"High School": female_result[6],"Some H.S.":female_result[7],
        "Less than H.S.":female_result[8], "None":female_result[9]},
        "Male":{"Doctorate": male_result[0],"Professional": male_result[1],
        "Master's": male_result[2],"Bachelor's":male_result[3],"Associate's":male_result[4],
        "Some College":male_result[5],"High School": male_result[6],"Some H.S.":male_result[7],
        "Less than H.S.":male_result[8], "None":male_result[9]}}
    else:
        map = {"Female":{"Doctorate": "Not applicable","Professional": "Not applicable",
        "Master's": "Not applicable","Bachelor's":"Not applicable","Associate's":"Not applicable",
        "Some College":"Not applicable","High School": "Not applicable","Some H.S.":"Not applicable",
        "Less than H.S.":"Not applicable", "None":"Not applicable"},
        "Male":{"Doctorate": "Not applicable","Professional": "Not applicable",
        "Master's": "Not applicable","Bachelor's":"Not applicable","Associate's":"Not applicable",
        "Some College":"Not applicable","High School": "Not applicable","Some H.S.":"Not applicable",
        "Less than H.S.":"Not applicable", "None":"Not applicable"}}
    return(map)

In [15]:
# get detailed eductaional attainment information regard sex for all counties
def get_all_Detailed_Educational_Attainment_Sex_Ratio(url_list, index = 5):
    i=0
    headersF = ['County','Doctorate F','Professional F','Masters F','Bachelor F','Associate F','Some College F','High School F','Some HS F','Less than HS F','None F']
    headersM = ['County','Doctorate M','Professional M','Masters M','Bachelor M','Associate M','Some College M','High School M','Some HS M','Less than HS M','None M']
    dff = pd.DataFrame(columns=headersF)
    dfm = pd.DataFrame(columns=headersM)
    for a_list in url_list:
        county_name = a_list.split("/")[index]
        print(county_name + ":")
        Cmap = get_each_Detailed_Educational_Attainment_Sex_Ratio(get_soup(a_list))
        print(Cmap)
        print()
        print("\n")
        county = [county_name]
        mf = list(Cmap.values())
        female = list(mf[0].values())
        male = list(mf[1].values())
        combF = county + female
        combM = county + male
        dff.loc[i] = combF
        dfm.loc[i] = combM
        i += 1
    return dff,dfm

In [16]:
# get bachelor degree information broken down by age for each county
# parameter is result returned from get_soup()
def get_each_Bachelor_Degree_by_Age(results):
    result = []
    i = 0
    for r in results:
        if i < 91 and "%" in r.text.strip():
            i = i + 1
        elif i < 95 and "%" in r.text.strip():
            result.append(r.text.strip())
            i = i + 1
    if len(result) is not 0:
        map = {"65+":result[0],"45-64":result[1],"35-44":result[2],"25-34":result[3]}
    else:
        map = {"65+":"Not applicable","45-64":"Not applicable","35-44":"Not applicable","25-34": "Not applicable"}
    return(map)

In [17]:
# get bachelor degree information broken down by age for all counties
def get_all_Bachelor_Degree_by_Age(url_list, index = 5):
    i=0
    headers = ['County','65+','45-64','35-44','25-34']
    df = pd.DataFrame(columns=headers)
    for a_list in url_list:
        county_name = a_list.split("/")[index]
        print(county_name + ":")
        valmap = get_each_Bachelor_Degree_by_Age(get_soup(a_list))
        print(valmap)
        print("\n")
        county = [county_name]
        mapvalue = list(valmap.values())
        combList = county + mapvalue
        print(combList)
        df.loc[i] = combList
        i += 1
    return df

In [ ]:
# save all returned data into csv file
savedData0 = get_all_Eductional_Attainment(get_url_list())
savedData0.to_csv('files/AllEduAttainment.csv', sep=',', encoding='utf-8')

savedData1 = get_all_Detailed_Educational_Attainment(get_url_list())
savedData1.to_csv('files/AllDetailedEduAttainment.csv', sep=',', encoding='utf-8')

savedData2 = get_all_Educational_Attainment_Sex_Ratio(get_url_list())
savedData2.to_csv('files/AllEduAttainmentSexRatio.csv', sep=',', encoding='utf-8')

savedData2f, savedData2m = get_all_Detailed_Educational_Attainment_Sex_Ratio(get_url_list())
savedData2f.to_csv('files/ADEASexRatioFemale.csv', sep=',', encoding='utf-8')
savedData2m.to_csv('files/ADEASexRatioMale.csv', sep=',', encoding='utf-8')

savedData3 = get_all_Bachelor_Degree_by_Age(get_url_list())
savedData3.to_csv('files/AllBachDegreeByAge.csv', sep=',', encoding='utf-8')

### This part is used for zip code level data scraping

#### Since our project is focused on county level analysis because of the time limit, the following part can be used in futher development